In [43]:
from gremlin_python.driver.aiohttp.transport import AiohttpTransport
from graph_search.graph_search import GraphSearch, __
from gremlin_python.process.traversal import P, TextP, Bytecode, Cardinality
from gremlin_python.process.translator import Order


graph_search = GraphSearch( "ws://localhost:8182/gremlin", 
                    transport_factory=lambda:AiohttpTransport(call_from_event_loop=True)
                    )


g = graph_search.graph

========message_serializer <gremlin_python.driver.serializer.GraphSONMessageSerializer object at 0x107eb1ac0>
Graphsearch is using gremlinpython==3.7.1


In [44]:
g.V().has('code', 'BNA').limit(1).element_map().toList()

[<Node:airport id="4112" date= country=US code=BNA longest=11030 city=Nashville author= lon=-86.6781997680664 type=airport elev=599 icao=KBNA region=US-TN runways=4 lat=36.1245002746582 desc=Nashville International Airport >]

In [45]:
# graph_search.graph.V().has('code', 'BNA').inE().toList()

# get nodes, out_e, in_e
g.V().has('code', 'BNA').as_('node') \
.project('nodes','out_e', 'in_e') \
.by(__.as_("node")) \
.by(__.outE()) \
.by(__.inE()) \
.to_list()

[{'nodes': <Node:airport id="4112" date= country=US code=BNA longest=11030 city=Nashville author= lon=-86.6781997680664 type=airport elev=599 icao=KBNA region=US-TN runways=4 lat=36.1245002746582 desc=Nashville International Airport >,
  'out_e': <RelationShip:route id="1q802-368-f11-38o" 4112:airport -> route -> 4200:airport dist=630 >,
  'in_e': <RelationShip:route id="1t6od-38o-f11-368" 4200:airport -> route -> 4112:airport dist=630 >}]

In [46]:
# get nodes and their `outE` counts 
data = g.V().project('v','e') \
.by(__.id_()).by(__.outE().count()) \
.order().by("e", Order.desc).limit(5).toList()


print(data)
# for d in data:
#     print(d)

[{'v': 1556600, 'e': 989}, {'v': 1445904, 'e': 971}, {'v': 1593544, 'e': 604}, {'v': 1429584, 'e': 586}, {'v': 3096648, 'e': 321}]


In [47]:
# get nodes and if there are any `outE` edges 
g.V().limit(2) \
.project('v','e') \
.by(__.id_()).by(__.coalesce(__.outE().limit(1).constant(True),__.constant(False))).toList()

[{'v': 4112, 'e': True}, {'v': 8208, 'e': True}]

In [48]:
# get all outE relationships of Npde
g.V().hasId(4112).has('code', 'BNA').outE().inV().path().by(__.elementMap()).limit(5).toList()

[path[<Node:airport id="4112" date= country=US code=BNA longest=11030 city=Nashville author= lon=-86.6781997680664 type=airport elev=599 icao=KBNA region=US-TN runways=4 lat=36.1245002746582 desc=Nashville International Airport >, <RelationShip:route id="1q802-368-f11-38o" 4112:airport -> route -> 4200:airport dist=630 >, <Node:airport id="4200" date= country=US code=DFW longest=13401 city=Dallas author= lon=-97.0380020141602 type=airport elev=607 icao=KDFW region=US-TX runways=7 lat=32.896800994873 desc=Dallas/Fort Worth International Airport >],
 path[<Node:airport id="4112" date= country=US code=BNA longest=11030 city=Nashville author= lon=-86.6781997680664 type=airport elev=599 icao=KBNA region=US-TN runways=4 lat=36.1245002746582 desc=Nashville International Airport >, <RelationShip:route id="1q96q-368-f11-38w" 4112:airport -> route -> 4208:airport dist=657 >, <Node:airport id="4208" date= country=US code=IAH longest=12001 city=Houston author= lon=-95.3414001464844 type=airport el

In [49]:
# get all outE relationships of Npde
g.V().hasId(4112).as_("n").outE() \
.project('n','v','e') \
.by(__.select("n")) \
.by() \
.by(__.otherV()).limit(2).toList()

[{'n': <Node:airport id="4112" date= country=US code=BNA longest=11030 city=Nashville author= lon=-86.6781997680664 type=airport elev=599 icao=KBNA region=US-TN runways=4 lat=36.1245002746582 desc=Nashville International Airport >,
  'v': <RelationShip:route id="1q802-368-f11-38o" 4112:airport -> route -> 4200:airport dist=630 >,
  'e': <Node:airport id="4200" date= country=US code=DFW longest=13401 city=Dallas author= lon=-97.0380020141602 type=airport elev=607 icao=KDFW region=US-TX runways=7 lat=32.896800994873 desc=Dallas/Fort Worth International Airport >},
 {'n': <Node:airport id="4112" date= country=US code=BNA longest=11030 city=Nashville author= lon=-86.6781997680664 type=airport elev=599 icao=KBNA region=US-TN runways=4 lat=36.1245002746582 desc=Nashville International Airport >,
  'v': <RelationShip:route id="1q96q-368-f11-38w" 4112:airport -> route -> 4208:airport dist=657 >,
  'e': <Node:airport id="4208" date= country=US code=IAH longest=12001 city=Houston author= lon=-95

In [50]:
# filter by traversal conditions: relationships count
# g.V().hasLabel('airport').where(__.outE().count().is_(P.lt(1))).count().next()
g.V().hasLabel('airport').where(
    __.and_(
        __.outE().count().is_(P.gte(10)),
        __.outE().count().is_(P.lte(20))
    )
).count().next()


382

In [51]:
# get vertices, ordered by __.outE (desc)

g.V() \
.project('v','e') \
.by(__.id_()) \
.by( 
    __.outE().count().is_(P.lt(2))
)\
.order().by("e", Order.desc) \
.limit(10).select("v", "e").to_list()


[{'v': 143376, 'e': 1},
 {'v': 303120, 'e': 1},
 {'v': 307216, 'e': 1},
 {'v': 335888, 'e': 1},
 {'v': 376848, 'e': 1},
 {'v': 385040, 'e': 1},
 {'v': 417808, 'e': 1},
 {'v': 430096, 'e': 1},
 {'v': 438288, 'e': 1},
 {'v': 446480, 'e': 1}]

In [52]:
# filter by traversal conditions: relationships count
# g.V().hasLabel('airport').where(__.outE().count().is_(P.lt(1))).count().next()
g.V().hasLabel('airport').where(
    __.and_(
        __.outE().count().is_(P.gt(10)),
        __.and_(
            __.outE().count().is_(P.gte(10)),
            __.outE().count().is_(P.lte(20))
        )
    )
).count().next()


322

In [53]:
g.V().hasId(4112).has('code', 'BNA').element_map().to_list()
# .order().by("dist", Order.asc)

[<Node:airport id="4112" date= country=US code=BNA longest=11030 city=Nashville author= lon=-86.6781997680664 type=airport elev=599 icao=KBNA region=US-TN runways=4 lat=36.1245002746582 desc=Nashville International Airport >]